# Building and training VGG13

### Import libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms.functional as F
from torchvision.datasets import ImageFolder
from torchvision import transforms


###  Load, preprocess, analyze the dataset and make it ready for training

In [2]:
# Define transforms for dataset
transform = transforms.Compose(
    [transforms.Resize(256),
     transforms.CenterCrop(224),
     transforms.ToTensor()])

# Load the dataset
traindata = ImageFolder(root='./cnn_dataset', transform=None)

# Compute the mean and standard deviation values for your dataset
mean = torch.zeros(3)
std = torch.zeros(3)
for img, _ in traindata:
    img = F.to_tensor(img)
    mean += img.mean(dim=(1, 2))
    std += img.std(dim=(1, 2))

mean /= len(traindata)
std /= len(traindata)


# Normalize the dataset
normalize = transforms.Normalize(mean=mean, std=std)
traindata = ImageFolder(root='./cnn_dataset', transform=transforms.Compose([
    transform,
    normalize,
]))

# Split the dataset into train and validation sets
trainsize = int(0.8 * len(traindata))
testsize = len(traindata) - trainsize
traindata, testdata = torch.utils.data.random_split(traindata, [trainsize, testsize])

# Create dataloaders
train_loader = torch.utils.data.DataLoader(traindata, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(testdata, batch_size=64, shuffle=False)


### Implement VGG-13 model

In [3]:
class VGG13(nn.Module):
    def __init__(self,num_classes):
        super(VGG13, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 3),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


### Training the VGG13 model

In [4]:
import torch.optim as optim
import time

# Define the model
net = VGG13(num_classes=3)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Train the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
print('Device:', device)

train_losses = []
train_accs = []
test_losses = []
test_accs = []
training_time_list = []

for epoch in range(2):
    start_time = time.time()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    test_loss = 0.0
    test_correct = 0
    test_total = 0  
    
    # Train the model
    net.train()
    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        
    # Evaluate the model on the testing data
    net.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    epo_time = time.time() - start_time
    # Save the training and test loss and accuracy for this epoch
    train_losses.append(train_loss / len(train_loader))
    train_accs.append(train_correct / train_total)
    test_losses.append(test_loss / len(test_loader))
    test_accs.append(test_correct / test_total)
    training_time_list.append(epo_time)
    # Print the results for this epoch
    print(f"Epoch {epoch+1} -- Training Loss: {train_losses[-1]:.4f}, "
          f"Training Accuracy: {train_accs[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}, "
          f"Test Accuracy: {test_accs[-1]:.4f},  epo_time: {(epo_time/60):.3f} mins")




Device: cpu
Epoch 1 -- Training Loss: 1.0987, Training Accuracy: 0.3326, Test Loss: 1.0987, Test Accuracy: 0.3262,  epo_time: 267.520 mins
Epoch 2 -- Training Loss: 1.0987, Training Accuracy: 0.3326, Test Loss: 1.0987, Test Accuracy: 0.3262,  epo_time: 249.843 mins


### The model is trained for 2 epochs. Each epoch ran for 4 hours.